# Google Smartphone Decimeter Challenge_MyFirstTry by Neural Net
* This is my first try using Neural Net(Keras)
* Input of NN is (_derived.csv, baseline)
* I couldn't still understand the data about _derived.csv(https://www.kaggle.com/c/google-smartphone-decimeter-challenge/discussion/246424)
* Getting fucking score like 21263.733
  * I found some wrong point about creating submission file. Time and estimated values were set to wrong correspondence. Thats why I got fuckin score like  21263.733. See the NeuralNet().test()
  * Fixed this wrong point then I get the score 7.345, this is the start line I think.
* I will try to improve the score based on this code
  * Many people are using Kalman Filter as a Post processing.

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import gc
from contextlib import redirect_stdout

# Show Baseline Score

In [ ]:
baseline_train = pd.read_csv("../input/google-smartphone-decimeter-challenge/baseline_locations_train.csv")
baseline_test = pd.read_csv("../input/google-smartphone-decimeter-challenge/baseline_locations_test.csv")
submission = pd.read_csv("../input/google-smartphone-decimeter-challenge/sample_submission.csv")
# submission = submission.assign(latDeg=test.latDeg.round(6), lngDeg=test.lngDeg.round(6))

In [ ]:
import numpy as np

# evaluation
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle（円周） distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    緯度経度の2地点間の地球上の円周距離を計算（haversine formula）
    """
    RADIUS = 6_367_000 # 赤道半径：6_378_100[m], 極半径： 6_356_775[m], avg. = 6_367_437.5[m]
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

# percentile: 中央値や4分数の一般化、50パーセンタイル→中央値、95パーセンタイル→上位5%の区切りの値
def percentile50(x):
    return np.percentile(x, 50)

def percentile95(x):
    return np.percentile(x, 95)

def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [ ]:
# ground_truth
import pathlib
from tqdm.notebook import tqdm
INPUT = '../input/google-smartphone-decimeter-challenge'

p = pathlib.Path(INPUT)
gt_files = list(p.glob('train/*/*/ground_truth.csv'))
print('ground_truth.csv count : ', len(gt_files))

gts = []
for gt_file in tqdm(gt_files):
    gts.append(pd.read_csv(gt_file))
ground_truth = pd.concat(gts)

In [ ]:
print(get_train_score(baseline_train, ground_truth))

In [ ]:
def ansolute_error(df, gt):
    # calc_distance_error
    err_latDeg = abs(df["latDeg"] - gt["latDeg"])
    err_lngDeg = abs(df["lngDeg"] - gt["lngDeg"])
    return err_latDeg, err_lngDeg
#err_latDeg, err_lngDeg = ansolute_error(baseline_train, ground_truth)
#print(err_latDeg.mean(), err_lngDeg.mean())

In [ ]:
del baseline_train, baseline_test, submission
gc.collect()

# Feature Enginerrling

In [ ]:
import pandas as pd
import copy
import numpy as np
import pathlib
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from multiprocessing import Pool
import multiprocessing as multi


INPUT = '../input/google-smartphone-decimeter-challenge'
p = pathlib.Path(INPUT)

def ×××(self, df)　は思い付きの特徴量計算（大きすぎる値を小さくするなど）

def ×××2(self, df)は👆の結果を見て、ちゃんと整形したもの（忘れないように、以下にその過程を示す）

In [ ]:
def file_load_multi_pricessing(filename):
    df = pd.read_csv(filename)
    return df

In [ ]:
class FeatureEngineering:
    def __init__(self, debug_flag=True):
        if debug_flag:
            self.train_files = list(p.glob('train/*/*/*_derived.csv'))
            self.test_files = list(p.glob('test/*/*/*_derived.csv')) 
            self.raw_data = self.load_raw_data()
        else:
            pass
        
    def load_raw_data(self):
        # load all data, return merged train/test data
        result_train = []
        thread_num = multi.cpu_count()
        with Pool(thread_num) as pool:
            imap = pool.imap(file_load_multi_pricessing, self.train_files)
            result_train = list(tqdm(imap, total=len(self.train_files), desc="load train data"))
        train_data = pd.concat(result_train, ignore_index=True)

        result_test = []
        with Pool(4) as pool:
            imap = pool.imap(file_load_multi_pricessing, self.test_files)
            result_test = list(tqdm(imap, total=len(self.test_files), desc="load test data"))
        test_data = pd.concat(result_test, ignore_index=True)

        return pd.concat([train_data, test_data])

    def test_all(self):
        df = self.load_raw_data()        
        self.df = self.data_process(df)
    
    def data_process(self, df):
        assert len(df) == 6357741, "train, testに同様の変換が行われることを保証するために、２つのデータを１つのDataFrameにまとめてください。len(df) = 6357741行になるはずです。\n To guarantee train/test are same transformation, it requires combine train and test data to one DataFrame. It should be len(df)=6357741." 
        
        df = self.received_sv_time_nanos2(df)        
        df = self.raw_prm2(df)
        df = self.signal_type2(df)
        df = self.sat_pos2(df)
        df = self.sat_vel2(df)

        df = self.sat_clk_bias_m2(df)
        df = self.sat_clk_drift_mps2(df)
        df = self.raw_pr_unc_m2(df)
        df = self.isrb_m2(df)
        df = self.iono_delay_m2(df)
        df = self.tropo_delay_m2(df)
        df = self.constellation_type2(df)
        df = self.svid2(df)

        gc.collect()
        return df
    
    def received_sv_time_nanos(self, df):
        print("received_sv_time_nanos ... it takes a bit time")
        # 'receivedSvTimeInGpsNanos' - 'millisSinceGpsEpoch' = 衛星送信時間からスマホでの記録時間の差？millisSinceGpsEpoch ←こいつは何だ
        df['receivedSvTimeInGpsNanos_processed'] = 1000_000*df['millisSinceGpsEpoch'] - df['receivedSvTimeInGpsNanos'] # floatで計算できない大きな値なので、intで計算した後にfloatにする
        df['receivedSvTimeInGpsNanos_processed'] = 0.001*0.001*0.001*df['receivedSvTimeInGpsNanos_processed']  # [sec]にする

        return df
    
    def received_sv_time_nanos2(self, df):
        print("received_sv_time_nanos2")
        df = self.received_sv_time_nanos(df)
        target = df['receivedSvTimeInGpsNanos_processed']
        
        target = (target-1.075)/0.025
        target[-1 > target] = -1
        target[1 < target] = 1
        
        df['receivedSvTimeInGpsNanos_processed']  = target
        return df
    
    def raw_prm(self, df):
        print("raw_prm")
        param = 0.00000001
        df['rawPrM_processed'] = df['rawPrM'] * param 
        return df.drop(['rawPrM'], axis=1) 
    
    def raw_prm2(self, df):
        print("raw_prm_2")
        df = self.raw_prm(df)
        target = df['rawPrM_processed']
        
        target[target > 0.3] = 0.3
        target = (target - np.min(target))
        target = target / np.max(target)
        target = (target - 0.5) * 2
        
        df['rawPrM_processed'] = target
        return df
      
    def signal_type(self, df):
        print("signal_type")
        ret = []
        signal_list = ["GPS_L1", "GPS_L5", "GAL_E1", "GAL_E5A", "GLO_G1", "BDS_B1I", "BDS_B1C", "BDS_B2A", "QZS_J1", "QZS_J5"]
        for data in tqdm(df['signalType']):
            
            if "GPS_L1":
                if data in signal_list:
                    ret.append(signal_list.index(data)+1)
            else:
                ret.append(0)
            
        df['signalType_processed'] = ret
        return df.drop(['signalType'], axis=1)
    
    def signal_type2(self, df):
        # 0~1にする
        print("signal_type2")
        df = self.signal_type(df)
        df['signalType_processed'] = df['signalType_processed'] * 0.1
        return df

    
    def sat_pos(self, df):
        print("sat_pos")
        param = 10e-7
        df['xSatPosM_processed']  =  df['xSatPosM'] * param
        df['ySatPosM_processed']  =  df['ySatPosM'] * param
        df['zSatPosM_processed']  =  df['zSatPosM'] * param
        return df.drop(['xSatPosM', 'ySatPosM', 'zSatPosM'], axis=1)
    
    def sat_pos2(self, df):
        print("sat_pos2")
        #df = self.sat_pos(df)
        def func(target):
            target = (target - np.min(target))
            target = target / np.max(target)
            target = (target - 0.5) * 2
            return target
        
        df['xSatPosM_processed'] = func(df['xSatPosM'])
        df['ySatPosM_processed'] = func(df['ySatPosM'])
        df['zSatPosM_processed'] = func(df['zSatPosM'])
        return df.drop(['xSatPosM', 'ySatPosM', 'zSatPosM'], axis=1)

    def sat_vel(self, df):
        print("sat_pos")
        param = 10e-4
        df['xSatVelMps_processed']  =  df['xSatVelMps'] * param
        df['ySatVelMps_processed']  =  df['ySatVelMps'] * param
        df['zSatVelMps_processed']  =  df['zSatVelMps'] * param
        return df.drop(['xSatVelMps', 'ySatVelMps', 'zSatVelMps'], axis=1)
    
    def sat_vel2(self, df):
        print("sat_vel2")
        def func(target):
            target = (target - np.min(target))
            target = target / np.max(target)
            target = (target - 0.5) * 2
            return target
        
        df['xSatVelMps_processed'] = func(df['xSatVelMps'])
        df['ySatVelMps_processed'] = func(df['ySatVelMps'])
        df['zSatVelMps_processed'] = func(df['zSatVelMps'])
        return df.drop(['xSatVelMps', 'ySatVelMps', 'zSatVelMps'], axis=1)    

    def sat_clk_bias_m(self, df):
        print("sat_clk_bias_m")
        param = 10e-6
        df['satClkBiasM_processed'] = df['satClkBiasM'] * param 
        return df.drop(['satClkBiasM'], axis=1)
    
    def sat_clk_bias_m2(self, df):
        print("sat_clk_bias_m2")
        df = self.sat_clk_bias_m(df)
        target = df['satClkBiasM_processed']
        
        target[target > 5] = 5
        target[target < -5] = -5

        target = (target - np.min(target))
        target = target / np.max(target)
        target = (target - 0.5) * 2
        
        df['satClkBiasM_processed'] = target
        return df
    
    def sat_clk_drift_mps(self, df):
        print("sat_clk_drift_mps")
        param = 1.0
        df['satClkDriftMps_processed'] = df['satClkDriftMps'] * param 
        return df.drop(['satClkDriftMps'], axis=1)
    
    def sat_clk_drift_mps2(self, df):
        print("sat_clk_drift_mps2")
        df = self.sat_clk_drift_mps(df)
        target = df['satClkDriftMps_processed']
        
        target[target > 0.015] = 0.015
        target[target < -0.015] = -0.015

        target = (target - np.min(target))
        target = target / np.max(target)
        target = (target - 0.5) * 2
        
        df['satClkDriftMps_processed'] = target
        return df

    
    def raw_pr_unc_m(self, df):
        print("raw_pr_unc_m")
        param = 1.0
        df['rawPrUncM_processed'] = df['rawPrUncM'] * param 
        return df.drop(['rawPrUncM'], axis=1)

    def raw_pr_unc_m2(self, df):
        print("raw_pr_unc_m2")
        df = self.raw_pr_unc_m(df)
        target = df['rawPrUncM_processed']
        
        target[target > 26] = 25
        target[target < 0] = 0

        target = (target - np.min(target))
        target = target / np.max(target)
        target = (target - 0.5) * 2
        
        df['rawPrUncM_processed'] = target
        return df
    
    def isrb_m(self, df):
        print("isrb_m")
        param = 1.0
        df['isrbM_processed'] = df['isrbM'] * param 
        return df.drop(['isrbM'], axis=1)
    
    def isrb_m2(self, df):
        print("isrb_m2: don't use this value")
        return df.drop(['isrbM'], axis=1)
    
    def iono_delay_m(self, df):
        print("iono_delay_m")
        param = 1.0
        df['ionoDelayM_processed'] = df['ionoDelayM'] * param 
        return df.drop(['ionoDelayM'], axis=1)
    
    def iono_delay_m2(self, df):
        print("iono_delay_m2")
        df = self.iono_delay_m(df)
        target = df['ionoDelayM_processed']
        
        target[target > 21] = 21
        target[target < 2] = 2

        target = (target - np.min(target))
        target = target / np.max(target)
        target = (target - 0.5) * 2
        
        df['ionoDelayM_processed'] = target
        return df

    def tropo_delay_m(self, df):
        print("tropo_delay_m")
        param = 1.0
        df['tropoDelayM_processed'] = df['tropoDelayM'] * param 
        return df.drop(['tropoDelayM'], axis=1)
    
    def tropo_delay_m2(self, df):
        print("tropo_delay_m2")
        df = self.tropo_delay_m(df)
        target = df['tropoDelayM_processed']
        
        target[target > 30] = 30
        target[target < 2] = 2

        target = (target - np.min(target))
        target = target / np.max(target)
        target = (target - 0.5) * 2
        
        df['tropoDelayM_processed'] = target
        return df
    
    def constellation_type(self, df):
        print("constellation_type")
        return df
        
    def constellation_type2(self, df):
        print("constellation_type2")
        df['constellationType_processed'] = df['constellationType'] / max(df['constellationType'])
        return df.drop(['constellationType'], axis=1)
    
    def svid(self, df):
        print("svid")
        return df
    
    def svid2(self, df):
        print("svid2")
        df['svid_processed'] = df['svid'] / max(df['svid'])
        return df.drop(['svid'], axis=1)


# Data Analysis with Feature Engineering for Neural Net

In [ ]:
feature_engineering = FeatureEngineering()

### receivedSvTimeInGpsNanos(and millisSinceGpsEpoch)
チップセットが受信した信号の送信時間で、GPSエポック以降のナノ秒数で表します。ReceivedSvTimeNanosから変換すると、この派生値はすべての星座で統一された時間スケールになりますが、ReceivedSvTimeNanosはGLONASSでは一日の時間、GLONASS以外の星座では週の時間を参照します。

The signal transmission time received by the chipset, in the numbers of nanoseconds since the GPS epoch. Converted from ReceivedSvTimeNanos, this derived value is in a unified time scale for all constellations, while ReceivedSvTimeNanos refers to the time of day for GLONASS and the time of week for non-GLONASS constellations.

millisSinceGpsEpoch(GPSエポック(1980/1/6 midnight UTC)からのミリ秒単位の整数。)との差分をとって値を小さくする。どちらも大きい値なので、適当に小さくしたかった。この差分が何を意味するのかはいまいちわかってない（millisSinceGpsEpochが何を意味しているのかがいまいち掴めていない）

received_sv_time_nanos_result = param1\*receivedSvTimeInGpsNanos - param2\*millisSinceGpsEpoch

In [ ]:
data = feature_engineering.received_sv_time_nanos(feature_engineering.raw_data)["receivedSvTimeInGpsNanos_processed"]

plt.hist(data)

ヒストグラムがずいぶん偏っているので範囲を1.05~1.10に狭めてみてみる(この範囲は何回か試して決めた)

In [ ]:
target = data[(1.05 < data) & (data < 1.1)]
plt.hist(target, 50)
print(len(data))
print(len(target))
print(len(target)/len(data))

全データのうち約95%が1.05 ~ 1.10にある。

1.75が中央になるように変更、1.05未満を0、1.10以上を1にする

In [ ]:
plt.hist((target-1.075)/0.025, 50)

In [ ]:
p_data = data
p_data = (p_data-1.075)/0.025
p_data[-1 > p_data] = -1
p_data[1 < p_data] = 1

plt.hist(p_data, 50)

-> この結果からreceived_sv_time_nanos2を決めた

In [ ]:
data = feature_engineering.received_sv_time_nanos2(feature_engineering.raw_data)["receivedSvTimeInGpsNanos_processed"]
plt.hist(data, 50)
del data, p_data, target # to reduce using RAM
gc.collect()

### signalType

GNSS信号タイプは、コンステレーション名と周波数帯の組み合わせです。スマートフォンで測定される一般的な信号タイプは以下の通りです。GPS_L1、GPS_L5、GAL_E1、GAL_E5A、GLO_G1、BDS_B1I、BDS_B1C、BDS_B2A、QZS_J1、QZS_J5。

The GNSS signal type is a combination of the constellation name and the frequency band. Common signal types measured by smartphones include: GPS_L1, GPS_L5, GAL_E1, GAL_E5A, GLO_G1, BDS_B1I, BDS_B1C, BDS_B2A, QZS_J1, and QZS_J5.

**cf.    def signal_type(self, df):**    

| 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 |
| ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- |
| Other | GPS_L1 | GPS_L5 | GAL_E1 | GAL_E5A | GLO_G1 | BDS_B1I | BDS_B1C | BDS_B2A | QZS_J1 | QZS_J5 |

In [ ]:
feature_engineering.signal_type(feature_engineering.raw_data)

In [ ]:
signal_type_data = feature_engineering.signal_type(feature_engineering.raw_data)["signalType_processed"].value_counts()
print(signal_type_data)

OK

1が多くて10が少ない

0~1にする **<span style="color: red; ">本当はone-hot vectorにすべきだと思う</span>**

In [ ]:
signal_type_data = feature_engineering.signal_type2(feature_engineering.raw_data)["signalType_processed"].value_counts()
print(signal_type_data)

del signal_type_data # to reduce using RAM
gc.collect()

### rawPrM
メートル単位の生の疑似距離。これは、光速と、信号送信時刻(receivedSvTimeInGpsNanos)から信号到着時刻(Raw::TimeNanos - Raw::FullBiasNanos - Raw::BiasNanos)までの時間差の積である。

Raw pseudorange in meters. It is the product between the speed of light and the time difference from the signal transmission time (receivedSvTimeInGpsNanos) to the signal arrival time (Raw::TimeNanos - Raw::FullBiasNanos - Raw::BiasNanos).

**※ここではヘンテコ処理をした後なので値は小さくなっているが、ヒストグラムの形状は変わらないはず(This result is processed so the value is different from the original rawPrM but the shape of histgram is same.)**

**cf.     def raw_prm(self, df):**

In [ ]:
rawPrM_processed = feature_engineering.raw_prm(feature_engineering.raw_data)["rawPrM_processed"]
plt.hist(rawPrM_processed, 50)
print(len(rawPrM_processed))
print(len(rawPrM_processed[rawPrM_processed<0.3]))
print(len(rawPrM_processed[rawPrM_processed<0.3])/len(rawPrM_processed))

0.3未満に97%のデータが含まれているので、0.3以上はoutlierとして扱う

In [ ]:
target = rawPrM_processed
target[target > 0.3] = 0.3
plt.hist(target, 100)

-1~1に整形する

In [ ]:
target2 = (target - np.min(target))
target2 = target2 / np.max(target2)
target2 = (target2 - 0.5) * 2
plt.hist(target2, 100)

これをraw_prm2とする

In [ ]:
rawPrM_processed = feature_engineering.raw_prm2(feature_engineering.raw_data)["rawPrM_processed"]
plt.hist(rawPrM_processed, 50)

del rawPrM_processed, target2, target
gc.collect()

## [x/y/z]SatPosM
ttx = receivedSvTimeInGpsNanos - satClkBiasNanos (以下に定義)で定義される "真の信号送信時間 "の最良の推定値におけるECEF座標フレーム内の衛星位置(メートル)。これらは、衛星放送のエフェメリスを用いて計算され、真の衛星位置に対して約1mの誤差があります。

The satellite position (meters) in an ECEF coordinate frame at best estimate of “true signal transmission time” defined as ttx = receivedSvTimeInGpsNanos - satClkBiasNanos (defined below). They are computed with the satellite broadcast ephemeris, and have ~1-meter error with respect to the true satellite position.

In [ ]:
position_processed = feature_engineering.sat_pos(feature_engineering.raw_data)

x_position = position_processed["xSatPosM_processed"]
y_position = position_processed["ySatPosM_processed"]
z_position = position_processed["zSatPosM_processed"]

plt.figure(figsize=(34, 7), dpi=50)
plt.subplot(1,3,1)
plt.hist(x_position, 50) 
plt.subplot(1,3,2)
plt.hist(y_position, 50)
plt.subplot(1,3,3)
plt.hist(z_position, 50)
plt.show()

outlierはなさそうなので、このまま1~-1に整形する(sat_pos2を参照)

In [ ]:
position_processed = feature_engineering.sat_pos2(feature_engineering.raw_data)

x_position = position_processed["xSatPosM_processed"]
y_position = position_processed["ySatPosM_processed"]
z_position = position_processed["zSatPosM_processed"]

plt.figure(figsize=(34, 7), dpi=50)
plt.subplot(1,3,1)
plt.hist(x_position, 50) 
plt.subplot(1,3,2)
plt.hist(y_position, 50)
plt.subplot(1,3,3)
plt.hist(z_position, 50)
plt.show()

del position_processed, x_position, y_position, z_position
gc.collect()

## sat vel

In [ ]:
vel_processed = feature_engineering.sat_vel(feature_engineering.raw_data)

x_vel = vel_processed["xSatVelMps_processed"]
y_vel = vel_processed["ySatVelMps_processed"]
z_vel = vel_processed["zSatVelMps_processed"]

plt.figure(figsize=(34, 7), dpi=50)
plt.subplot(1,3,1)
plt.hist(x_vel, 50) 
plt.subplot(1,3,2)
plt.hist(y_vel, 50)
plt.subplot(1,3,3)
plt.hist(z_vel, 50)
plt.show()

outlierはなさそうなので、このまま1~-1に整形する(sat vel2を参照)

sat vel2

In [ ]:
vel_processed = feature_engineering.sat_vel2(feature_engineering.raw_data)

x_vel = vel_processed["xSatVelMps_processed"]
y_vel = vel_processed["ySatVelMps_processed"]
z_vel = vel_processed["zSatVelMps_processed"]

plt.figure(figsize=(34, 7), dpi=50)
plt.subplot(1,3,1)
plt.hist(x_vel, 50) 
plt.subplot(1,3,2)
plt.hist(y_vel, 50)
plt.subplot(1,3,3)
plt.hist(z_vel, 50)
plt.show()

del vel_processed, x_vel, y_vel, z_vel
gc.collect()

### Others - satClkBiasM	satClkDriftMps	rawPrUncM	isrbM	ionoDelayM	tropoDelayM	

In [ ]:
sat_clk_bias_m = feature_engineering.sat_clk_bias_m(feature_engineering.raw_data)['satClkBiasM_processed']
sat_clk_drift_mps = feature_engineering.sat_clk_drift_mps(feature_engineering.raw_data)['satClkDriftMps_processed']
raw_pr_unc_m = feature_engineering.raw_pr_unc_m(feature_engineering.raw_data)['rawPrUncM_processed']
isrb_m = feature_engineering.isrb_m(feature_engineering.raw_data)['isrbM_processed']
iono_delay_m = feature_engineering.iono_delay_m(feature_engineering.raw_data)['ionoDelayM_processed']
tropo_delay_m = feature_engineering.tropo_delay_m(feature_engineering.raw_data)['tropoDelayM_processed']

plt.figure(figsize=(34, 14), dpi=50)
plt.subplot(2,3,1)
plt.title("sat_clk_bias_m")
plt.hist(sat_clk_bias_m, 50) 

plt.subplot(2,3,2)
plt.title("sat_clk_drift_mps")
plt.hist(sat_clk_drift_mps, 50)

plt.subplot(2,3,3)
plt.title("raw_pr_unc_m")
plt.hist(raw_pr_unc_m, 50)

plt.subplot(2,3,4)
plt.title("isrb_m")
plt.hist(isrb_m, 50)

plt.subplot(2,3,5)
plt.title("iono_delay_m")
plt.hist(iono_delay_m, 50)

plt.subplot(2,3,6)
plt.title("tropo_delay_m")
plt.hist(tropo_delay_m, 50)
plt.show()


outlierと偏りがそれぞれあるので、１つづつ調整する

In [ ]:
plt.figure(figsize=(34, 14), dpi=50)
plt.subplot(2,3,1)
plt.title("sat_clk_bias_m(-5, 5)")
plt.hist(sat_clk_bias_m[(-5 < sat_clk_bias_m)&(sat_clk_bias_m<5)], 50) 
print("sat_clk_bias_mの-5 ~ 5に含まれる割合")
print(len(sat_clk_bias_m[(-5 < sat_clk_bias_m)&(sat_clk_bias_m<5)])/len(sat_clk_bias_m))

plt.subplot(2,3,2)
plt.title("sat_clk_drift_mps(-0.015, 0.015)")
plt.hist(sat_clk_drift_mps[(-0.015 < sat_clk_drift_mps)&(sat_clk_drift_mps<0.015)], 50)
print("sat_clk_drift_mps -0.015 ~ 0.015に含まれる割合")
print(len(sat_clk_drift_mps[(-0.015 < sat_clk_drift_mps)&(sat_clk_drift_mps<0.015)])/len(sat_clk_drift_mps))

plt.subplot(2,3,3)
plt.title("raw_pr_unc_m(0,25)")
plt.hist(raw_pr_unc_m[(0 < raw_pr_unc_m)&(raw_pr_unc_m<25)], 50)
print("raw_pr_unc_m 0 ~ 25に含まれる割合")
print(len(raw_pr_unc_m[(0 < raw_pr_unc_m)&(raw_pr_unc_m<25)])/len(raw_pr_unc_m))

plt.subplot(2,3,4)
plt.title("isrb_m")
plt.hist(isrb_m[(-0.001 < isrb_m)&(isrb_m<0.001)], 50)
print("isrb_m: 傾向が見えないので、後で別処理")

plt.subplot(2,3,5)
plt.title("iono_delay_m")
plt.hist(iono_delay_m[(2 < iono_delay_m)&(iono_delay_m<21)], 100)
print("iono_delay_m 2 ~ 21に含まれる割合")
print(len(iono_delay_m[(1 < iono_delay_m)&(iono_delay_m<21)])/len(iono_delay_m))

plt.subplot(2,3,6)
plt.title("tropo_delay_m")
plt.hist(tropo_delay_m[(2 < tropo_delay_m)&(tropo_delay_m<30)], 100)
print("tropo_delay_m 2 ~ 30に含まれる割合")
print(len(tropo_delay_m[(2 < tropo_delay_m)&(tropo_delay_m<30)])/len(tropo_delay_m))
plt.show()

In [ ]:
isrb_m.value_counts()

isrb_mはほとんど0

ノイズにしかならなそうなので、削除？

In [ ]:
sat_clk_bias_m = feature_engineering.sat_clk_bias_m2(feature_engineering.raw_data)['satClkBiasM_processed']
sat_clk_drift_mps = feature_engineering.sat_clk_drift_mps2(feature_engineering.raw_data)['satClkDriftMps_processed']
raw_pr_unc_m = feature_engineering.raw_pr_unc_m2(feature_engineering.raw_data)['rawPrUncM_processed']
#isrb_m = feature_engineering.isrb_m2(feature_engineering.raw_data)['isrbM_processed']
iono_delay_m = feature_engineering.iono_delay_m2(feature_engineering.raw_data)['ionoDelayM_processed']
tropo_delay_m = feature_engineering.tropo_delay_m2(feature_engineering.raw_data)['tropoDelayM_processed']

plt.figure(figsize=(34, 14), dpi=50)
plt.subplot(2,3,1)
plt.title("sat_clk_bias_m")
plt.hist(sat_clk_bias_m, 50) 

plt.subplot(2,3,2)
plt.title("sat_clk_drift_mps")
plt.hist(sat_clk_drift_mps, 50)

plt.subplot(2,3,3)
plt.title("raw_pr_unc_m")
plt.hist(raw_pr_unc_m, 50)

#plt.subplot(2,3,4)
#plt.title("isrb_m")
#plt.hist(isrb_m, 50)

plt.subplot(2,3,5)
plt.title("iono_delay_m")
plt.hist(iono_delay_m, 50)

plt.subplot(2,3,6)
plt.title("tropo_delay_m")
plt.hist(tropo_delay_m, 50)
plt.show()

In [ ]:
del sat_clk_bias_m, sat_clk_drift_mps, raw_pr_unc_m, isrb_m, iono_delay_m, tropo_delay_m
gc.collect()

### constellationType, svid

In [ ]:
constellationType = feature_engineering.constellation_type2(feature_engineering.raw_data)['constellationType_processed']
svid = feature_engineering.svid2(feature_engineering.raw_data)['svid_processed']

print(constellationType.value_counts())
print(svid.value_counts())

del constellationType, svid
gc.collect()

## test all

In [ ]:
feature_engineering.test_all()
display(feature_engineering.df)

In [ ]:
del feature_engineering
gc.collect()

In [ ]:
# check memory: https://qiita.com/AnchorBlues/items/883790e43417640140aa
import sys

print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

# Neural Net

## data loader

In [ ]:
import pandas as pd
import copy
import numpy as np
import pathlib
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import os
import glob
from multiprocessing import Pool
import multiprocessing as multi


INPUT = '../input/google-smartphone-decimeter-challenge'
p = pathlib.Path(INPUT)

In [ ]:
def file_load_multi_pricessing(filename):
    df = pd.read_csv(filename)
    return df

In [ ]:
class DataLoader:
    def __init__(self):
        self.train_files = list(p.glob('train/*/*/*_derived.csv'))
        self.test_files = list(p.glob('test/*/*/*_derived.csv'))
    
        self.feature_engineering = FeatureEngineering(debug_flag=False)
        
        raw_data = self.load_raw_data() 
        
        raw_data = self.feature_engineering.data_process(raw_data)
        
        # train/test split
        train_data = raw_data[raw_data["train_flag"]==1]
        test_data = raw_data[raw_data["train_flag"]==0]
        
        del raw_data
        gc.collect()
        
        # add ground truth to train data
        gt_files = list(p.glob('train/*/*/ground_truth.csv'))
        data=[]
        for filename in tqdm(gt_files, desc="Loading ground truth"):
            df = pd.read_csv(filename)
            data.append(df)
        train_gt = pd.concat(data, ignore_index=True)

        train_gt["receivedSvTimeInGpsNanos"] = train_gt.millisSinceGpsEpoch*int(1e6)
        train_data = train_data.drop("millisSinceGpsEpoch", axis=1)
        
        train_gt_data = pd.merge_asof(train_data.sort_values('receivedSvTimeInGpsNanos'), train_gt.sort_values('receivedSvTimeInGpsNanos'), 
                                                   on="receivedSvTimeInGpsNanos", by=["collectionName", "phoneName"], direction='nearest',tolerance=int(1e9))
        self.train_gt_data = train_gt_data.sort_values(by=["collectionName", "phoneName", "millisSinceGpsEpoch"], ignore_index=True)
        
        # add baseline value
        # merge with baseline data
        baseline_train = pd.read_csv("../input/google-smartphone-decimeter-challenge/baseline_locations_train.csv")
        baseline_test = pd.read_csv("../input/google-smartphone-decimeter-challenge/baseline_locations_test.csv")


        self.train_gt_data = pd.merge_asof(self.train_gt_data.sort_values('millisSinceGpsEpoch'), baseline_train.sort_values('millisSinceGpsEpoch'), 
                                                           on="millisSinceGpsEpoch", suffixes=("_gt", "_base"),  by=["collectionName", "phoneName"], direction='nearest',tolerance=int(1e9))
        self.test_data = pd.merge_asof(test_data.sort_values('millisSinceGpsEpoch'), baseline_test.sort_values('millisSinceGpsEpoch'), 
                                                           on="millisSinceGpsEpoch", suffixes=("_gt", "_base"),  by=["collectionName", "phoneName"], direction='nearest',tolerance=int(1e9))

        self.train_gt_data = self.gt_ajust(self.train_gt_data)
        #self.train_gt_data, self.test_data, self.lat_bias, self.lng_bias = self.lat_lng_ajust(self.train_gt_data, self.test_data)

    def load_raw_data(self):
        # load all data, return merged train/test data
        result_train = []
        thread_num = multi.cpu_count()
        with Pool(thread_num) as pool:
            imap = pool.imap(file_load_multi_pricessing, self.train_files)
            result_train = list(tqdm(imap, total=len(self.train_files), desc="load train data"))
        train_data_ = pd.concat(result_train, ignore_index=True)
        train_data = pd.DataFrame([{"train_flag":1}]*len(train_data_)).join([train_data_])

        result_test = []
        with Pool(4) as pool:
            imap = pool.imap(file_load_multi_pricessing, self.test_files)
            result_test = list(tqdm(imap, total=len(self.test_files), desc="load test data"))
        test_data = pd.concat(result_test, ignore_index=True)
        test_data = pd.DataFrame([{"train_flag":0}]*len(test_data)).join([test_data])
        
        return pd.concat([train_data, test_data])
        
    def load_one_data(self, df):
        ret_data = []
        unique_time = sorted(df["millisSinceGpsEpoch"].unique())
        for target_time in unique_time:
            tmp_df = df[df["millisSinceGpsEpoch"]==target_time]
            ret_data.append(copy.deepcopy(tmp_df))
            break # debug
        return ret_data
    
    def lat_lng_ajust(self, df_train, df_test):
        lat_bias = df_train['latDeg_gt'].mean()
        lng_bias = df_train['lngDeg_gt'].mean()
        
        df_train['latDeg_gt'] = df_train['latDeg_gt'] - lat_bias
        df_train['lngDeg_gt'] = df_train['lngDeg_gt'] - lng_bias
        df_train['latDeg_base'] = df_train['latDeg_base'] - lat_bias
        df_train['lngDeg_base'] = df_train['lngDeg_base'] - lng_bias

        # same transformation with train
        df_test['latDeg'] = df_test['latDeg'] - lat_bias
        df_test['lngDeg'] = df_test['lngDeg'] - lng_bias
        return df_train, df_test, lat_bias, lng_bias
    
    def gt_ajust(self, df_train):
        df_train['latDeg_gt2'] = df_train['latDeg_gt'] - df_train['latDeg_base']
        df_train['lngDeg_gt2'] = df_train['lngDeg_gt'] - df_train['lngDeg_base']
        return df_train

In [ ]:
gc.collect()

In [ ]:
data_loader = DataLoader()

In [ ]:
df = data_loader.train_gt_data[['latDeg_gt', 'lngDeg_gt', 'latDeg_base', 'lngDeg_base']]
print(df.max())
print(df.min())

In [ ]:
df = data_loader.test_data[['latDeg', 'lngDeg']]
print(df.max())
print(df.min())

In [ ]:
data_loader.train_gt_data

In [ ]:
data_loader.test_data

In [ ]:
data_loader.train_gt_data.columns

In [ ]:
del data_loader
gc.collect()

## Training and Test(submission)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale

import os
from contextlib import redirect_stdout

# 評価指標
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import log_loss

# keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.core import Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.layers import Input, Dense

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint

In [ ]:
'''
all_feature_train = ['train_flag', 'collectionName', 'phoneName', 'receivedSvTimeInGpsNanos',
                   'receivedSvTimeInGpsNanos_processed', 'rawPrM_processed',
                   'signalType_processed', 'xSatPosM_processed', 'ySatPosM_processed',
                   'zSatPosM_processed', 'xSatVelMps_processed', 'ySatVelMps_processed',
                   'zSatVelMps_processed', 'satClkBiasM_processed',
                   'satClkDriftMps_processed', 'rawPrUncM_processed',
                   'ionoDelayM_processed', 'tropoDelayM_processed',
                   'constellationType_processed', 'svid_processed', 'millisSinceGpsEpoch',
                   'latDeg_gt', 'lngDeg_gt', 'heightAboveWgs84EllipsoidM_gt',
                   'timeSinceFirstFixSeconds', 'hDop', 'vDop', 'speedMps', 'courseDegree',
                   'latDeg_base', 'lngDeg_base', 'heightAboveWgs84EllipsoidM_base',
                   'phone']
'''

use_feature_train = ['receivedSvTimeInGpsNanos_processed', 'rawPrM_processed',
           'signalType_processed', 'xSatPosM_processed', 'ySatPosM_processed',
           'zSatPosM_processed', 'xSatVelMps_processed', 'ySatVelMps_processed',
           'zSatVelMps_processed', 'satClkBiasM_processed',
           'satClkDriftMps_processed', 'rawPrUncM_processed',
           'ionoDelayM_processed', 'tropoDelayM_processed',
           'constellationType_processed', 'svid_processed',
           'latDeg_base', 'lngDeg_base']

use_feature_test = ['receivedSvTimeInGpsNanos_processed', 'rawPrM_processed',
   'signalType_processed', 'xSatPosM_processed', 'ySatPosM_processed',
   'zSatPosM_processed', 'xSatVelMps_processed', 'ySatVelMps_processed',
   'zSatVelMps_processed', 'satClkBiasM_processed',
   'satClkDriftMps_processed', 'rawPrUncM_processed',
   'ionoDelayM_processed', 'tropoDelayM_processed',
   'constellationType_processed', 'svid_processed',
   'latDeg', 'lngDeg']

nn_data_param_length = len(use_feature_train)


assert_error_str = "must be same length with use_feature_train="+str(len(use_feature_train))+" and use_feature_test="+str(len(use_feature_test))
assert nn_data_param_length == len(use_feature_test), assert_error_str
                                                                                                                          

In [ ]:
class NeuralNet:
    def __init__(self, data_loader):
        
        print("split to x/y")
        self.train_data_x = data_loader.train_gt_data[use_feature_train]
        self.train_data_y = data_loader.train_gt_data[['latDeg_gt2', 'lngDeg_gt2']]
        
        self.test_data_x = data_loader.test_data[use_feature_test]
        
        self.x_train, self.x_val, self.y_train, self.y_val = train_test_split(self.train_data_x, self.train_data_y, test_size=0.1, random_state=40, shuffle=False)        


        
    
    ###########################################################################################################
    #
    #    model 
    #
    #########################################################################################################    
    
    
    def create_model_simple(self):
        activation_fun = "swish"
        
        inputs = keras.Input(shape=(nn_data_param_length,))
        x = inputs
                
        x = layers.Dense(512, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)

        x = layers.Dense(512, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
        x = layers.Dense(512, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
        x = layers.Dense(512, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        
        outputs = layers.Dense(2, activation="linear")(x)
        return keras.Model(inputs, outputs)
    
    def create_model_self_attention(self):
        activation_fun = "swish"
        
        inputs = keras.Input(shape=(nn_data_param_length,))
        x = inputs
        
        query_encoding = x
        query_value_attention = tf.keras.layers.Attention()([x, x])
        x = tf.keras.layers.Concatenate()([query_encoding, query_value_attention])
        x = layers.Dense(256, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
        x = layers.Dense(512, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
        x_ = x
        
        x = layers.Dense(512, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
        query_encoding = x
        query_value_attention = tf.keras.layers.Attention()([x, x])
        x = tf.keras.layers.Concatenate()([query_encoding, query_value_attention])
        x = layers.Dense(512, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
        x = layers.Dense(512, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
        x = keras.layers.Concatenate()([x, x_])
        x = layers.Dense(512, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
        x = layers.Dense(512, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        
        x = layers.Dense(256, activation=activation_fun, kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        
        outputs = layers.Dense(2, activation="linear")(x)
        return keras.Model(inputs, outputs)
    
    ###########################################################################################################
    #
    #    training & test
    #
    #########################################################################################################   
    
    def calculate_loss(self, ground_truth, pred):
        loss = pd.DataFrame({'LogLoss': log_loss(ground_truth, pred)},
                               index = ['scores'])
        return loss
    
    def calculate_scores(self, ground_truth, pred):
        scores = pd.DataFrame({'Accuracy': accuracy_score(ground_truth, pred),
                              'F1': f1_score(ground_truth, pred),
                              'Precision': precision_score(ground_truth, pred),
                              'Recall': recall_score(ground_truth, pred)},
                               index = ['scores'])
        return scores
    
    def haversine_my_metrics(self, y_true, y_pred):
        
        lat1, lon1, lat2, lon2 = y_true[0], y_true[1], y_pred[0], y_pred[1]

        RADIUS = 6_367_000 # 赤道半径：6_378_100[m], 極半径： 6_356_775[m], avg. = 6_367_437.5[m]
        def deg2rad(deg):
            pi_on_180 = 0.017453292519943295
            return deg * pi_on_180
        lat1, lon1, lat2, lon2 = map(deg2rad, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = tf.sin(dlat/2)**2 + \
            tf.cos(lat1) * tf.cos(lat2) * tf.sin(dlon/2)**2
        dist = 2 * RADIUS * tf.asin(a**0.5)
        return dist
        
    def test(self):
        self.model_compile()
        self.model.load_weights('./best_weights.hdf5')
        self.result = self.model.predict(self.test_data_x, batch_size=2048)
        
        result = pd.DataFrame(self.result, columns=["latDeg", "lngDeg"])
        result["latDeg"] = result["latDeg"] + self.test_data_x["latDeg"]
        result["lngDeg"] = result["lngDeg"] + self.test_data_x["lngDeg"]

        tmp = data_loader.test_data[['phone', 'millisSinceGpsEpoch']]
        test_result = tmp.join(result)        
        test_result = test_result.groupby(["phone", "millisSinceGpsEpoch"]).mean().reset_index()

        sample_df = pd.read_csv("../input/google-smartphone-decimeter-challenge/sample_submission.csv")
        sample_df = sample_df.drop(["latDeg", "lngDeg"], axis=1)
        self.submission = pd.merge_asof(sample_df.sort_values('millisSinceGpsEpoch'), 
                                    test_result.sort_values('millisSinceGpsEpoch'), 
                                    on="millisSinceGpsEpoch", by=["phone"], direction='nearest', tolerance=100000)
        
        self.submission = sample_df.merge(self.submission) # sort to the submission order
        self.submission.to_csv("my_submission.csv", index=False)
        display(self.submission)
        
    def model_compile(self):
        #self.model = self.create_model_self_attention()
        self.model = self.create_model_simple()
        lr_schedule = keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=1e-2,
            decay_steps=10000,
            decay_rate=0.9)
        self.optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
        self.model.compile(loss='huber', optimizer=self.optimizer, metrics=["mae", self.haversine_my_metrics])
        
    def train(self, print_result_flag=False):
        self.model_compile()
        cp = ModelCheckpoint("best_weights.hdf5", monitor="val_mae", verbose=1,
             save_best_only=True, save_weights_only=True, mode="min")
        self.model.fit(self.x_train, self.y_train, epochs=10, batch_size=2048, verbose=2, callbacks=[cp], validation_data=(self.x_val, self.y_val))

    def evaluate_train(self):
        self.model_compile()
        self.model.load_weights('./best_weights.hdf5')
        
        train_score = self.model.evaluate(self.x_train, self.y_train, batch_size=2048)
        val_score = self.model.evaluate(self.x_val, self.y_val, batch_size=2048)
        
        print(f"train_score: {train_score}, val_score: {val_score}")
        
    def evaluate_train2(self):
        self.model_compile()
        self.model.load_weights('./best_weights.hdf5')
        result = self.model.predict(self.train_data_x, batch_size=2048)
        
        self.train_result = pd.DataFrame(result, columns=['latDeg', 'lngDeg'])
        
        self.train_result = self.train_result.rename(columns={"latDeg": "latDeg_original", "lngDeg": "lngDeg_original"})
        self.train_result["latDeg"] = self.train_result["latDeg_original"] + self.train_data_x["latDeg_base"]
        self.train_result["lngDeg"] = self.train_result["lngDeg_original"] + self.train_data_x["lngDeg_base"]
        err_latDeg, err_lngDeg = ansolute_error(self.train_result, ground_truth)
        print(err_latDeg.mean(), err_lngDeg.mean())        
        
        tmp = data_loader.train_gt_data[['collectionName', 'phoneName', 'millisSinceGpsEpoch']]
        train_result = tmp.join(self.train_result)
        print(get_train_score(train_result, ground_truth))

In [ ]:
data_loader = DataLoader()

In [ ]:
gc.collect()
my_nn = NeuralNet(data_loader)
gc.collect()

baseline absolute error: lat = 0.10239727214110347, lng = 0.1432227609733908

In [ ]:
my_nn.train()

In [ ]:
my_nn.evaluate_train()

In [ ]:
my_nn.evaluate_train2()

In [ ]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

baseline absolute error: lat = 0.10239727214110347, lng = 0.1432227609733908

5.287970649084159

In [ ]:
my_nn.test()